In [20]:
#pip install sqlalchemy
#pip install pyodbc

import pandas as pd
import sqlalchemy as sal

In [21]:
engine=sal.create_engine(r"mssql://KALIVARUN\MSSQL/master?driver=ODBC+DRIVER+17+FOR+SQL+SERVER")
conn= engine.connect()

In [38]:
def extract():
    df_products=pd.read_csv("products.txt")
    df_products_db=pd.read_sql_query("select * from products",conn)
    return df_products,df_products_db


def transform(df_products,df_products_db):
    df_merge=pd.merge(df_products,df_products_db,how="left",left_on = "product_id",right_on="product_id")
    df_insert=df_merge[df_merge["product_name_y"].isna()]
    df_insert_final=df_insert.iloc[:,0:3]
    df_insert_final.columns=df_products_db.columns

    df_update=df_merge[df_merge["product_name_y"].notna()]
    df_update_final=df_update.iloc[:,0:3]
    df_update_final.columns=df_products_db.columns
    return df_insert_final,df_update_final


def load_staging(df_update_final):
    df_update_final.to_sql("products_stg",con=conn,index=False,if_exists="replace")
    conn.commit()

def updates():
    query=sal.text("update products set price = products_stg.price, product_name=products_stg.product_name from products_stg where products.product_id=products_stg.product_id")
    p=conn.execute(query)
    conn.commit()

def inserts(df_insert_final):
    df_insert_final.to_sql("products",con=conn,index=False,if_exists="append")
    conn.commit()


    
    

    

In [39]:
df_products,df_products_db= extract()

df_insert_final,df_update_final =transform(df_products,df_products_db)

load_staging(df_update_final)

inserts(df_insert_final)

updates()

In [36]:
def extract():
    df_products = pd.read_csv('products.txt')
    df_products_db = pd.read_sql_query('select * from products' , conn)
    return df_products,df_products_db

def transform(df_products,df_products_db):
    df_merged = pd.merge(df_products , df_products_db , how='left' , on = 'product_id')
    df_insert = df_merged[df_merged['product_name_y'].isna()]
    df_insert_final =  df_insert.iloc[: , 0:3]
    df_insert_final.columns = df_products_db.columns
    
    df_update = df_merged[df_merged['product_name_y'].notna()]
    df_update_final =  df_update.iloc[: , 0:3]
    df_update_final.columns = df_products_db.columns
    return df_insert_final,df_update_final

def load_staging(df_update_final):
    df_update_final.to_sql('products_stg',con=conn , index=False , if_exists = 'replace')
    conn.commit()

def updates():
    query = sal.text("update products set price = products_stg.price, product_name=products_stg.product_name from products_stg where products.product_id=products_stg.product_id")
    p = conn.execute(query)
    conn.commit()
    
def inserts(df_insert_final):
    df_insert_final.to_sql('products',con=conn , index=False , if_exists = 'append')
    conn.commit()

In [37]:
df_products,df_products_db = extract()

df_insert_final,df_update_final = transform(df_products,df_products_db)

load_staging(df_update_final)

inserts(df_insert_final)

updates()